In [1]:
pip install transformers


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Akash\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

C:\Users\Akash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Akash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Akash\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/do

In [32]:
from datasets import load_dataset
import pandas as pd
import re

# Original categories
category_to_label = {
    0: 'Billing Escalation',
    1: 'Technical Issue Escalation',
    2: 'Product Issue Escalation',
    3: 'Customer Service Escalation',
    4: 'Technology Issue Escalation',
    5: 'User Experience Escalation',
    6: 'Legal Escalation',
    7: 'Project Issue Escalation',
    8: 'R&D Escalation',
    9: 'Promotions Escalation'
}



# Mapping function to clean and normalize category names
def clean_category(category):
    category = category.lower()  # Lowercase all characters
    category = re.sub(r'\s+', ' ', category)  # Normalize multiple spaces to one space
    category = re.sub(r'[^a-zA-Z0-9 ]', '', category)  # Remove special characters
    return category


cleaned_categories = {}
for idx, cat in category_to_label.items():
    cleaned_name = clean_category(cat)
    cleaned_categories[cleaned_name] = cat

def map_category_to_cleaned_category(category, cleaned_categories):
    cleaned_input = clean_category(category)  # Normalize the input category
    for cleaned_key, original_category in cleaned_categories.items():
        if cleaned_key in cleaned_input:  # If the cleaned category matches the input
            return original_category
    return 'Unknown'



def convert_to_json_format_with_text_labels(dataset):
    """
    Convert a Hugging Face dataset with 'email_body', 'subject', and 'category' columns into the desired JSON format.
    It also encodes categorical labels into numeric form.
    
    Args:
    - dataset (datasets.Dataset): A Hugging Face dataset with 'email_body', 'subject', and 'category' columns.
    
    Returns:
    - List of dictionaries in the desired format.
    """
    # Convert Hugging Face dataset to pandas DataFrame
    df = pd.DataFrame(dataset)

        # Create a label encoding for the 'category' column (text labels to numeric)
    label_to_category = {v: k for k, v in category_to_label.items()}  # Reverse mapping (numeric label to category)
    df['label'] = df['category'].apply(lambda x: label_to_category.get(map_category_to_cleaned_category(x, cleaned_categories)))

    result = []

    # Iterate through the dataframe rows
    for _, row in df.iterrows():
        # Combine 'subject' and 'email_body' to form the 'text'
        text = row['subject'] + " " + row['email_body']
        
        if(pd.isna(row['label'])):
            continue

        # Create the result in the desired JSON format
        item = {
            "label": row['label'],
            "text": text
        }
        
        result.append(item)
    
    return result, category_to_label, label_to_category

dataset = load_dataset("akashkpatel1991/escalation_synthetic_data", split="train")

json_data, category_to_label, label_to_category = convert_to_json_format_with_text_labels(dataset)


In [39]:
from datasets import Dataset

for item in json_data:
    item['label'] = int(item['label'])


dataset = Dataset.from_dict({
    'label': [item['label'] for item in json_data],
    'text': [item['text'] for item in json_data]
})

In [40]:
dataset[0]

{'label': 0,
 'text': 'Urgent: Billing Escalation – Immediate Attention Required Dear Company X Support Team,\\n\\nWe are escalating a billing discrepancy reported by customer John Doe. The customer has noticed an overcharge on their account for the last billing cycle, which has not yet been resolved. Please investigate and provide a resolution at the earliest.\\n\\nAgent Information:\\n- Agent Name: Emily Adams\\n- Agent ID: EA34567\\n\\nKindly rectify the charges and issue a refund if applicable.\\n\\nBest regards, Telus Customer Support Team'}

In [56]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map: 100%|██████████| 3806/3806 [00:00<00:00, 5488.55 examples/s]


In [ ]:
tokenized_dataset[0]

In [43]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.Collecting evaluate
   ---------------------------------------- 0.0/84.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.0 kB ? eta -:--:--
   ---------------------------------------- 84.0/84.0 kB 1.2 MB/s eta 0:00:00




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Akash\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [80]:
from transformers import DataCollatorWithPadding
import evaluate

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [49]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(category_to_label),
                                                           id2label=category_to_label, label2id=label_to_category)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
dataset_split = tokenized_dataset.train_test_split(test_size=0.2)  # 80% for training, 20% for testing

#  = dataset.train_test_split(test_size=0.2)  # 80% for training, 20% for testing


In [58]:
dataset_split['test']

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 762
})

In [82]:
training_args = TrainingArguments(
    output_dir="fine_tuned_esclation_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    #num_train_epochs=1,
    max_steps = 3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_split["train"],
    eval_dataset=dataset_split["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [83]:
trainer.train()

Epoch,Training Loss,Validation Loss


TypeError: EvaluationModule.compute() takes 1 positional argument but 3 were given